In [ ]:
!pip install openai

In [ ]:
from google.colab import userdata
import os

# Carga la clave API de los secretos de Colab
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
from openai import OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [ ]:
def get_gpt4o_response(full_prompt_text, temperature=0.1):
    """
    Envía un prompt a GPT-4o y devuelve la respuesta.
    full_prompt_text: El prompt completo a enviar, incluyendo la cláusula.
    temperature: La temperatura para la generación (más baja para mayor determinismo).
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a specialized legal assistant in Intellectual Property Law."},
                {"role": "user", "content": full_prompt_text}
            ],
            temperature=temperature,
            max_tokens=700 # Ajustado para respuestas detalladas
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error calling GPT-4o API: {e}")
        return "ERROR: Fallo al obtener respuesta de GPT-4o."

In [ ]:
# --- 20 Cláusulas Contractuales de Cesión de Derechos de Autor (Reutilizadas) ---
# Se asume que este listado de 'clauses' es idéntico al usado en el experimento de Gemma.
# Para evitar duplicación, puedes cargar este listado desde un archivo si lo guardaste antes.
# Por simplicidad aquí, lo volvemos a definir.
clauses = [
    {"id": "clausula_01", "text": "El Autor cede a la Editorial el derecho exclusivo de reproducción de la Obra por un período de cinco (5) años en todo el territorio nacional español, con una compensación del 10% sobre las ventas netas."},
    {"id": "clausula_02", "text": "El Artista cede a la Galería el derecho no exclusivo de exhibición pública de la Obra visual por tiempo indefinido en la Unión Europea, sin percibir compensación económica directa."},
    {"id": "clausula_03", "text": "El Compositor cede a la Productora el derecho exclusivo de comunicación pública (incluida la radiodifusión) de la Composición por un período de tres (3) años en Estados Unidos de América, con un pago único de 5.000 euros."},
    {"id": "clausula_04", "text": "El Desarrollador cede a la Empresa el derecho exclusivo de distribución del Software en formato físico por diez (10) años a nivel mundial, recibiendo un 25% de los beneficios brutos."},
    {"id": "clausula_05", "text": "El Fotógrafo cede a la Agencia el derecho no exclusivo de reproducción y comunicación pública de las Imágenes por un período de un (1) año en territorio español, mediante una tarifa plana mensual de 200 euros."},
    {"id": "clausula_06", "text": "El Autor cede todos los derechos patrimoniales sobre la Obra por tiempo indefinido y en todo el universo, sin compensación alguna más allá de la mención de autoría."},
    {"id": "clausula_07", "text": "El Creador cede a la Plataforma el derecho no exclusivo de puesta a disposición interactiva de los Contenidos por dos (2) años para Latinoamérica, con una compensación del 50% de los ingresos publicitarios generados."},
    {"id": "clausula_08", "text": "El Diseñador cede a la Marca el derecho exclusivo de transformación y distribución del Diseño por quince (15) años en Europa y Norteamérica, a cambio de un pago inicial de 10.000 dólares."},
    {"id": "clausula_09", "text": "El Guionista cede a la Productora el derecho exclusivo de adaptación cinematográfica del Guion por siete (7) años en territorio mundial (con exclusión de China), con un porcentaje sobre la taquilla bruta del 1%."},
    {"id": "clausula_10", "text": "El Investigador cede a la Universidad el derecho no exclusivo de reproducción y comunicación pública de la Publicación por tiempo limitado a la vida del autor más 70 años en cualquier país con normativa de PI, sin remuneración específica."},
    {"id": "clausula_11", "text": "La Empresa XYZ cede a la Filial ABC el derecho exclusivo de reproducción de su material de marketing por tres (3) años en Alemania y Francia, con una compensación fija de 2.500 euros anuales."},
    {"id": "clausula_12", "text": "El Titular del Software cede a Licenciatario el derecho no exclusivo de distribución del Software a través de plataformas digitales por tiempo indefinido en Asia (excluyendo Japón), con un pago por cada descarga de 0,50 dólares."},
    {"id": "clausula_13", "text": "El Compositor A cede al Editor B el derecho exclusivo de sincronización de la Música por cuatro (4) años en territorio de la Unión Europea, mediante un adelanto de 1.000 libras y 50% de regalías."},
    {"id": "clausula_14", "text": "El Creador de Contenido cede a la Red Social el derecho no exclusivo de comunicación pública y transformación de sus publicaciones por tiempo indefinido en todo el mundo, sin compensación adicional por su uso."},
    {"id": "clausula_15", "text": "La Cía. Farmacéutica cede a la Empresa de Biotecnología el derecho exclusivo de reproducción de sus resultados de investigación por quince (15) años en América Latina, con una licencia fija de 50.000 dólares."},
    {"id": "clausula_16", "text": "El Podcast Host cede a la Distribuidora el derecho no exclusivo de distribución y puesta a disposición de los Episodios por diez (10) años en Canadá y Australia, a cambio de un porcentaje del 5% de los ingresos por suscripciones."},
    {"id": "clausula_17", "text": "El Escultor cede a la Fundación el derecho de exhibición pública de la Obra escultórica por un período de veinte (20) años en museos específicos de Nueva York (EE. UU.), sin fines de lucro y sin compensación directa."},
    {"id": "clausula_18", "text": "La Universidad cede al Consorcio de Investigación el derecho no exclusivo de reproducción de artículos científicos para fines educativos por tiempo indefinido en cualquier país miembro de la OCDE, sin contraprestación económica."},
    {"id": "clausula_19", "text": "El Productor de Video cede a la Empresa de Medios el derecho exclusivo de comunicación pública y distribución del Contenido de Video por ocho (8) años en Estados Unidos y México, a cambio de un pago por capítulo emitido de 1.000 dólares."},
    {"id": "clausula_20", "text": "El Autor cede todos los derechos de explotación sobre su Novela (reproducción, distribución, comunicación pública y transformación) por toda la duración de los derechos según la LPI en todo el mundo, con un adelanto de 3.000 euros y 8% de royalties."}
]

# --- Prompt Base para Extracción de Parámetros (Idéntico al usado para Gemma) ---
PROMPT_BASE = """
Eres un asistente jurídico especializado en Derecho de Propiedad Intelectual. Tu tarea es analizar la siguiente cláusula contractual de cesión de derechos de autor.

**Instrucciones Clave:**
1.  **Extrae** únicamente la información explícita o directamente deducible de la cláusula. No realices interpretaciones jurídicas complejas ni añadas información que no esté directamente contenida en el texto.
2.  **Identifica** los cuatro parámetros fundamentales de la cesión:
    * **Tipo de Derecho Cedido:** ¿Qué derechos específicos se mencionan (ej. reproducción, distribución, comunicación pública, transformación, exhibición, adaptación, etc.) o se hace referencia a un conjunto de derechos (ej. "todos los derechos patrimoniales", "derechos de explotación")?
    * **Duración de la Cesión:** ¿Cuál es el plazo temporal de la cesión (ej. número de años, "tiempo indefinido", "vida del autor + 70 años", "toda la duración de los derechos")?
    * **Ámbito Territorial:** ¿Cuál es la delimitación geográfica específica de la cesión (ej. país, región, mundial, universo)?
    * **Forma de Compensación:** ¿Cómo se remunera la cesión (ej. pago único, porcentaje de ventas/beneficios, tarifa mensual/anual, sin compensación)?
3.  **Formato de Salida:** Presenta la información en un formato claro de lista, y para cada parámetro, indica **explícitamente la parte del texto de la cláusula** que fundamenta tu extracción.

**Cláusula a Analizar:**
[TEXTO DE LA CLÁUSULA]

**Ejemplo de Formato de Salida Deseado:**
- Tipo de Derecho Cedido: [Respuesta] (Texto fuente: "[Fragmento exacto de la cláusula]")
- Duración de la Cesión: [Respuesta] (Texto fuente: "[Fragmento exacto de la cláusula]")
- Ámbito Territorial: [Respuesta] (Texto fuente: "[Fragmento exacto de la cláusula]")
- Forma de Compensación: [Respuesta] (Texto fuente: "[Fragmento exacto de la cláusula]")
"""

In [ ]:
def get_gpt4o_response(clause_text, prompt_template=PROMPT_BASE, temperature=0.1):
    """
    Envía un prompt a GPT-4o para la extracción de parámetros de una cláusula
    y devuelve la respuesta generada.

    Args:
        clause_text (str): El texto de la cláusula a analizar.
        prompt_template (str): La plantilla del prompt base.
        temperature (float): La temperatura para la generación (0.0 para determinismo, >0.0 para creatividad).

    Returns:
        str: La respuesta generada por el modelo.
    """
    full_prompt_text = prompt_template.replace("[TEXTO DE LA CLÁUSULA]", clause_text)

    try:
        response = client.chat.completions.create(
            model="gpt-4o", # Modelo específico de OpenAI
            messages=[
                {"role": "system", "content": "You are a specialized legal assistant in Intellectual Property Law."},
                {"role": "user", "content": full_prompt_text}
            ],
            temperature=temperature,
            max_tokens=700, # Ajustado para respuestas detalladas
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error al obtener respuesta de GPT-4o para la cláusula: {clause_text[:50]}... Error: {e}")
        return "ERROR: Fallo al obtener respuesta de GPT-4o."

In [ ]:
import time
import json # Para guardar los resultados en un formato estructurado

results_gpt4o = []
output_filename_json = "gpt4o_experiment_5_1_1_results.json"

print("Iniciando generación de respuestas con GPT-4o...")
for i, clause_data in enumerate(clauses):
    clause_id = clause_data["id"]
    clause_text = clause_data["text"]

    print(f"\n--- Procesando {clause_id} ({i+1}/{len(clauses)}) ---")

    # Generar respuesta de GPT-4o
    # Usamos 0.1 para una alta precisión en la extracción descriptiva
    gpt4o_response = get_gpt4o_response(clause_text, temperature=0.1)

    # Almacenar el resultado
    results_gpt4o.append({
        "clause_id": clause_id,
        "clause_text": clause_text,
        "model_name": "GPT-4o",
        "prompt_used": PROMPT_BASE.replace("[TEXTO DE LA CLÁUSULA]", clause_text), # Guarda el prompt completo para trazabilidad
        "model_response": gpt4o_response
    })
    print(f"Respuesta de GPT-4o para {clause_id} generada.")
    # Se recomienda una pequeña pausa entre llamadas API para evitar límites de tasa
    time.sleep(0.5)

print("\nGeneración de respuestas de GPT-4o completada.")

# --- Guardar los resultados en un archivo JSON ---
with open(output_filename_json, 'w', encoding='utf-8') as f:
    json.dump(results_gpt4o, f, ensure_ascii=False, indent=4)

print(f"Resultados guardados en {output_filename_json}")

# --- Mostrar un ejemplo de resultado (opcional) ---
if results_gpt4o:
    print("\n--- Ejemplo de un resultado de GPT-4o ---")
    print(json.dumps(results_gpt4o[0], indent=4, ensure_ascii=False))

Iniciando generación de respuestas con GPT-4o...

--- Procesando clausula_01 (1/20) ---
Error al obtener respuesta de GPT-4o para la cláusula: El Autor cede a la Editorial el derecho exclusivo ... Error: name 'client' is not defined
Respuesta de GPT-4o para clausula_01 generada.

--- Procesando clausula_02 (2/20) ---
Error al obtener respuesta de GPT-4o para la cláusula: El Artista cede a la Galería el derecho no exclusi... Error: name 'client' is not defined
Respuesta de GPT-4o para clausula_02 generada.

--- Procesando clausula_03 (3/20) ---
Error al obtener respuesta de GPT-4o para la cláusula: El Compositor cede a la Productora el derecho excl... Error: name 'client' is not defined
Respuesta de GPT-4o para clausula_03 generada.

--- Procesando clausula_04 (4/20) ---
Error al obtener respuesta de GPT-4o para la cláusula: El Desarrollador cede a la Empresa el derecho excl... Error: name 'client' is not defined
Respuesta de GPT-4o para clausula_04 generada.

--- Procesando clausula_05

In [ ]:
import json
import os
import time # Para incluir la fecha y hora de generación del informe

# --- Configuración de Nombres de Archivo ---
json_filename_gpt4o = "gpt4o_experiment_5_1_1_results.json"
output_markdown_filename_gpt4o = "resultados_experimento_gpt4o_clausulas_PI.md"

# --- Paso 1: Cargar los resultados desde el archivo JSON ---
results_gpt4o = []
if os.path.exists(json_filename_gpt4o):
    with open(json_filename_gpt4o, 'r', encoding='utf-8') as f:
        results_gpt4o = json.load(f)
    print(f"[Cargados {len(results_gpt4o)} resultados desde '{json_filename_gpt4o}']")
else:
    print(f"Error: El archivo '{json_filename_gpt4o}' no se encontró. Asegúrate de que el experimento con GPT-4o se haya ejecutado y el archivo exista en tu entorno de Colab.")
    # Si el archivo no existe, no hay datos que procesar. El script se detendrá aquí.
    exit()

# --- Paso 2: Generar el contenido en formato Markdown ---
markdown_content_gpt4o = []

# Encabezado del documento Markdown
markdown_content_gpt4o.append("# Informe de Extracción de Parámetros de Cláusulas de Cesión de Derechos de Autor (Modelo GPT-4o)")
markdown_content_gpt4o.append("## Experimento 5.1.1: Recuperación de Cláusulas Contractuales de Cesión de Derechos de Autor")
markdown_content_gpt4o.append("---")
markdown_content_gpt4o.append("Este documento presenta las respuestas generadas por el modelo de Inteligencia Artificial **GPT-4o** al procesar una serie de cláusulas contractuales de cesión de derechos de autor. La tarea asignada al modelo fue la extracción de parámetros específicos (Tipo de Derecho Cedido, Duración, Ámbito Territorial y Forma de Compensación) sin realizar interpretaciones jurídicas complejas. Este informe está diseñado para facilitar la revisión y evaluación por parte de juristas para el cálculo del *Recall* y el LIAS.\n")
markdown_content_gpt4o.append(f"**Fecha de generación del informe:** {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())} (hora local)")
markdown_content_gpt4o.append(f"**Número total de cláusulas procesadas:** {len(results_gpt4o)}\n")
markdown_content_gpt4o.append("---")

# Iterar sobre cada resultado y formatearlo para Markdown
for entry in results_gpt4o:
    clause_id = entry.get("clause_id", "ID Desconocido")
    clause_text = entry.get("clause_text", "Texto de cláusula no disponible.")
    model_name = entry.get("model_name", "Modelo Desconocido")
    model_response = entry.get("model_response", "Respuesta del modelo no disponible.")
    # Puedes incluir el 'prompt_used' si consideras que es relevante para la revisión del abogado
    # prompt_used = entry.get("prompt_used", "Prompt no disponible.")

    markdown_content_gpt4o.append(f"\n## Cláusula: {clause_id}\n")
    markdown_content_gpt4o.append("### Texto Original de la Cláusula:\n")
    markdown_content_gpt4o.append(f"```\n{clause_text}\n```\n") # Usar bloque de código para el texto original y mantener formato

    markdown_content_gpt4o.append(f"### Respuesta del Modelo ({model_name}):\n")
    markdown_content_gpt4o.append(f"```\n{model_response}\n```\n") # Usar bloque de código para la respuesta del modelo

    # Si decidiste incluir el prompt:
    markdown_content_gpt4o.append("#### Prompt Completo Enviado:\n")
    # markdown_content_gpt4o.append(f"```\n{prompt_used}\n```\n")

    markdown_content_gpt4o.append("---\n") # Separador visual entre cláusulas

# --- Paso 3: Guardar el contenido Markdown en un archivo ---
try:
    with open(output_markdown_filename_gpt4o, 'w', encoding='utf-8') as f:
        f.write("\n".join(markdown_content_gpt4o))
    print(f"[Archivo '{output_markdown_filename_gpt4o}' generado con éxito. Contiene {len(results_gpt4o)} cláusulas formateadas.]")
except Exception as e:
    print(f"Error al escribir el archivo Markdown: {e}")

[Cargados 20 resultados desde 'gpt4o_experiment_5_1_1_results.json']
[Archivo 'resultados_experimento_gpt4o_clausulas_PI.md' generado con éxito. Contiene 20 cláusulas formateadas.]
